In [1]:
import pandas as pd
import numpy as np
import os

### Etapa 1 - Criação do dataframe com dados populacionais por ano dos municípios brasileiros para servir de base de cáclulo das variáveis [QUANTIDADE DE PROCEDIMENTOS POR 10 MIL HAB] E [VALORES DE PROCEDIEMNTOS POR 10 MIL HAB]

In [2]:
# Criando função para concatenar poulação dos municípios por ano, entre 2010 e 2022, a partir dos dados em arquivo csv na pasta ./data/demografico/ para uso em cálculo posterior
def load_population_data(folder_path):
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            year = int(filename.split('.')[0])
            df = pd.read_csv(os.path.join(folder_path, filename), sep=';', names=['codigo', 'pop'], dtype={'codigo': str})
            df.set_index('codigo', inplace=True)
            df = df[~df.index.duplicated(keep='first')] # remove possíveis linhas duplicadas
            df.rename(columns={'pop': str(year)}, inplace=True)
            dfs.append(df.iloc[1:, :]) # ignora a primeira linha
    merged_df = pd.concat(dfs, axis=1)
    merged_df.fillna(0, inplace=True) # preenche valores nulos com 0
    return merged_df


In [3]:
# Aplicando a função fornecendo a pasta com dados como parâmetro
df_pop = load_population_data('E:/OneDrive/Documentos/GitHub/iris/data/demografico')


In [4]:
# checando o dataframe e o resultado da função de concatenação de arquivos csv
df_pop

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1100015,24392,24228,24069,25728,25652,25578,25506,25437,23167,22945,22728,22516,22516
1100023,90353,91570,92747,101269,102860,104401,105896,107345,106168,107863,109523,111148,111148
1100031,6313,6221,6132,6495,6424,6355,6289,6224,5438,5312,5188,5067,5067
1100049,78574,78958,79330,85863,86556,87226,87877,88507,84813,85359,85893,86416,86416
1100056,17029,16939,16852,18041,18013,17986,17959,17934,16444,16323,16204,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,12548,12644,12737,13227,13343,13456,13567,13675,13746,13863,13977,14088,14088
5222054,7371,7475,7576,7933,8053,8171,8286,8397,8611,8743,8873,9002,9002
5222203,4735,4846,4954,5246,5371,5495,5615,5731,6026,6171,6312,6451,6451


In [5]:
# Checando quantidade de municípios
len(df_pop)

5570

In [6]:
# checando tipo de variáveis resultante da função 
df_pop.dtypes

2010    object
2011    object
2012    object
2013    object
2014    object
2015    object
2016    object
2017    object
2018    object
2019    object
2020    object
2021    object
2022    object
dtype: object

In [7]:
# convertendo dados para tipo integer
df_pop = df_pop.astype(int)

In [8]:
# Checando valores iguais a zero em cada coluna
num_zeros_por_coluna = (df_pop == 0).sum()
print(num_zeros_por_coluna)
del num_zeros_por_coluna # exclui a var temp, após verificação

2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    0
2019    0
2020    0
2021    0
2022    0
dtype: int64


Nota importante: cinco municípios dos 5.570 existentes atualmente não constavam nas bases de dados do IBGE entre 2010 e 2012, provavelmente foram criados após essa data. Os municípios tem os seguintes códigos: 1504752, 4212650, 4220000, 4314548, 5006275. Os dados inexistenets foram preenchidos com cálculo de população decrescente de 2012, para 2011 e para 2010 com abse na diferença da população anual dos anos subsequentes. Desse modo a matriz não resultará em dados nulos quando for utilizada para cálculos com outras matrizes para obtenção de dados relacionados à população, como é o caso da quantidade de procedimentos por 10 mil habitantes. 


In [9]:
import requests
# obtendo dados dos municípios listados por meio da API do IBGE

def obter_dados_municipios(codigos):
    dados_municipios = []
    for codigo in codigos.split("|"):
        try:
            url = f'https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{codigo}'
            response = requests.get(url)
            if response.status_code == 200:
                dados_municipio = response.json()
                nome_municipio = dados_municipio['nome']
                uf_municipio = dados_municipio['microrregiao']['mesorregiao']['UF']['sigla']
                dados_municipios.append({'Nome': nome_municipio, 'UF': uf_municipio})
        except requests.exceptions.RequestException as e:
            print(f"Erro ao obter dados do município {codigo}: {e}")
    return dados_municipios



In [10]:
# fornece parâmetros com os códigos dos municípios para obtenção do nome e UF
codigos = '1504752|4212650|4220000|4314548|5006275'
municipios = obter_dados_municipios(codigos)
for municipio in municipios:
    print(f"Nome: {municipio['Nome']}, UF: {municipio['UF']}")


Nome: Mojuí dos Campos, UF: PA
Nome: Pescaria Brava, UF: SC
Nome: Balneário Rincão, UF: SC
Nome: Pinto Bandeira, UF: RS
Nome: Paraíso das Águas, UF: MS


3 criados em 2013, sendo que Balneário Rincão e Pinto Bandeira foram emancipados antes de 2010. De toda forma os dados de população para os 5 municípios nos 3 anos foram projetados de maneira a não causar viés nos dados.

In [11]:
# mostando os dados dos municípios e as projeções resultantes para população nos anos de 2010 a 2012. 
df_pop.loc[['1504752', '4212650', '4220000', '4314548', '5006275']]


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
codigo,,,,,,,,,,,,,
1504752,14905,15014,15123,15232,15341,15446,15548,15646,15982,16084,16184,16282,16282
4212650,9465,9539,9613,9687,9761,9835,9908,9980,10022,10091,10159,10225,10225
4220000,11040,11236,11432,11628,11824,12018,12212,12403,12570,12760,12946,13129,13129
4314548,2324,2443,2562,2681,2800,2824,2847,2868,2968,3003,3036,3068,3068
5006275,4627,4732,4837,4942,5047,5150,5251,5350,5455,5555,5654,5751,5751


### Etapa 2 - Criação do dataframe com dados dos procedimentos para calcular matrizes de treinamento da rede neural autoencoder mlp e dos algoritmos de ML supervisionado e não supervisionado

In [12]:
df_sia = pd.read_csv('./data/SIA_procedimentos.csv', sep='|')
df_sia

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,co_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,329,201105,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108006,ACOMPANHAMENTO INTENSIVO DE PACIENTE EM SAUDE ...,166,3004.60
1,329,201102,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,52,772.20
2,329,201111,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,30,445.50
3,329,201209,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108010,ACOMPANHAMENTO NAO INTENSIVO DE PACIENTE EM SA...,42,623.70
4,329,201104,CAPS ESTACAO CIDADANIA,PREFEITURA MUNICIPAL DO CABO DE SAN,PE,260290.0,CABO DE SANTO AGOSTINHO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,110,1749.00
...,...,...,...,...,...,...,...,...,...,...,...
7477277,9993827,202009,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60480001,"ENOXAPARINA SÓDICA 40 MG/0,4 ML SOLUÇÃO INJETÁ...",30,0.00
7477278,9993827,202110,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477279,9993827,202208,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483001,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 50 MG (...,56,193.76
7477280,9993827,202207,FARMACIA CIDADA ESTADUAL DE GUACUI,SECRETARIA DE ESTADO DA SAUDE,ES,320230.0,GUACUI,60483002,SACUBITRIL VALSARTANA SÓDICA HIDRATADA 100 MG ...,60,207.60


In [13]:
df_sia.rename(columns={'co_procedimento': 'codigo_procedimento'}, inplace=True)

In [14]:
df_sih = pd.read_csv('./data/SIH_procedimentos.csv', sep='|')
df_sih

,cnes,competencia,estabelecimento,razao_social,uf,codigo_municipio,municipio,codigo_procedimento,procedimento,qtde_procedimentos,valor_procedimentos
0,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303080078,TRATAMENTO DE ESTREPTOCOCCIAS,11,340.62
1,35,201002,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303090316,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,11,238.70
2,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303020083,TRATAMENTO DE OUTRAS DOENÇAS DO SANGUE E DOS Ó...,24,388.37
3,35,201003,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,409060135,HISTERECTOMIA TOTAL,12,658.03
4,35,201004,HOSPITAL MENDO SAMPAIO,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,PE,260290,CABO DE SANTO AGOSTINHO,303070102,TRATAMENTO DE OUTRAS DOENCAS DO APARELHO DIGES...,15,380.58
...,...,...,...,...,...,...,...,...,...,...,...
28657071,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040099,HERNIOPLASTIA INGUINAL (BILATERAL),27,5490.54
28657072,9950931,202211,HOSPITAL MUNICIPAL INTEGRADO SANTO AMARO,PREFEITURA DO MUNICIPIO DE SAO PAULO,SP,355030,SAO PAULO,407040102,HERNIOPLASTIA INGUINAL / CRURAL (UNILATERAL),51,10845.49
28657073,9994289,202105,POLICLINICA MUNICIPAL ESTEFANIA FARIAS,PREFEITURA MUNICIPAL DE SURUBIM,PE,261450,SURUBIM,303010223,TRATAMENTO DE INFECÇÃO PELO CORONAVIRUS – COVI...,44,33000.00
28657074,9997784,202107,HOSPITAL E MATERNIDADE NAILA GONCALO,MUNICIPIO DE BACABAEIRA,MA,210125,BACABEIRA,411010034,PARTO CESARIANO,74,4867.57


In [15]:
df_sia['fonte'] = 'sia'
df_sih['fonte'] = 'sih'

In [16]:
# Definindo a opção "float_format" para exibir somente 1 dígito após a vírgula
pd.set_option("float_format", "{:.1f}".format)

In [17]:
df_sia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7477282 entries, 0 to 7477281
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 684.6+ MB


In [18]:
df_sih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28657076 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     int64  
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(1), int64(5), object(6)
memory usage: 2.6+ GB


In [19]:
df_procedimentos = pd.concat([df_sia, df_sih])
df_procedimentos.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36134358 entries, 0 to 28657075
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   cnes                 int64  
 1   competencia          int64  
 2   estabelecimento      object 
 3   razao_social         object 
 4   uf                   object 
 5   codigo_municipio     float64
 6   municipio            object 
 7   codigo_procedimento  int64  
 8   procedimento         object 
 9   qtde_procedimentos   int64  
 10  valor_procedimentos  float64
 11  fonte                object 
dtypes: float64(2), int64(4), object(6)
memory usage: 3.5+ GB


In [20]:
# cria um df com base em uma pivot table para os dados de quantidade de procedimento por cnes, com colunas com dados do município
df_qtde_proc_cnes = (
    df_procedimentos
    .pivot_table(index=['codigo_municipio', 'municipio', 'uf', 'cnes', 'codigo_procedimento', 'procedimento'], columns='competencia', values='qtde_procedimentos')
    .reset_index()
    .sort_index()
)

In [21]:
# Convertendo o índice "date" em uma coluna
df_qtde_proc_cnes = df_qtde_proc_cnes.reset_index()

# Removendo o nome da coluna "variable" da primeira linha
df_qtde_proc_cnes = df_qtde_proc_cnes.rename_axis(None, axis=1)
df_qtde_proc_cnes  = df_qtde_proc_cnes.drop('index', axis =1)

In [22]:
# modifica tipo de variável para codigo do município para inteiro
df_qtde_proc_cnes['codigo_municipio'] = df_qtde_proc_cnes['codigo_municipio'].astype(int)
df_qtde_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,301060010,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,55.0,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,301060088,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,94.0,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010010,TRATAMENTO DE DENGUE CLÁSSICA,139.0,113.0,30.0,19.0,...,12.0,6.0,28.0,138.0,261.0,189.0,91.0,134.0,183.0,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,...,4.0,22.0,12.0,56.0,157.0,NaN,63.0,249.0,2.0,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010045,TRATAMENTO DE DOENÇAS BACTERIANAS ZOONÓTICAS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377489,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101020,CADEIRA DE RODAS MONOBLOCO,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377490,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101021,CADEIRA DE RODAS (ACIMA 90KG),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377491,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101022,CADEIRA DE RODAS MOTORIZADA ADULTO OU INFANTIL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377492,530180,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,7988303,70101024,CADEIRA DE RODAS PARA BANHO COM ENCOSTO RECLIN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_qtde_proc_cnes.to_csv('./data/df_qtde_proc_cnes.csv', sep = '|', encoding = 'UTF-8')

In [23]:
# verifica a média de valores NAN mas colunas...  que se mostra muito alto, pois se temos um df com 1,37 milhões de linhas e a média de linhas com valores NaN é de 1,1 milhões, 
# então precisamos selecionar somente os procedimentos com poucos valores ausentes nas linhas tanto para CNES quanto apra municípios
df_qtde_proc_cnes.isna().sum().mean()

1103507.2654320989

In [24]:
df_qtde_proc_mun = (
    df_procedimentos
    .pivot_table(index=['codigo_municipio', 'municipio', 'uf', 'codigo_procedimento', 'procedimento'], columns='competencia', values='qtde_procedimentos')
    .reset_index()
    .sort_index()
)

In [25]:
# Convertendo o índice "date" em uma coluna
df_qtde_proc_mun = df_qtde_proc_mun.reset_index()

# Removendo o nome da coluna "variable" da primeira linha
df_qtde_proc_mun = df_qtde_proc_mun.rename_axis(None, axis=1)
df_qtde_proc_mun  = df_qtde_proc_mun.drop('index', axis =1)
df_qtde_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001.0,ALTA FLORESTA D'OESTE,RO,30108012,ACOMPANHAMENTO SEMI-INTENSIVO DE PACIENTES EM ...,NaN,168.0,168.0,768.0,840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110001.0,ALTA FLORESTA D'OESTE,RO,301060010,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,55.0,NaN
2,110001.0,ALTA FLORESTA D'OESTE,RO,301060088,DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,94.0,NaN
3,110001.0,ALTA FLORESTA D'OESTE,RO,303010010,TRATAMENTO DE DENGUE CLÁSSICA,139.0,113.0,30.0,19.0,4.0,...,12.0,6.0,28.0,138.0,261.0,189.0,91.0,134.0,183.0,NaN
4,110001.0,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,10.0,...,4.0,22.0,12.0,56.0,157.0,NaN,63.0,249.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932634,530180.0,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120032,MASTECTOMIA SIMPLES EM ONCOLOGIA,NaN,NaN,47.0,5.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932635,530180.0,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120040,RESSECÇÃO DE LESÃO NÃO PALPÁVEL DE MAMA COM MA...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932636,530180.0,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932637,530180.0,REGIAO ADMINISTRATIVA DE TAGUATINGA,DF,505010097,TRANSPLANTE DE CORNEA,54.0,48.0,72.0,33.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_qtde_proc_mun.to_csv('./data/df_qtde_proc_mun.csv', sep = '|', encoding = 'UTF-8')

In [26]:
df_qtde_proc_mun.isna().sum().mean()

728710.354037267

In [27]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_cnes.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
procedimentos_cnes_sem_nan = df_qtde_proc_cnes.dropna(thresh=min_valid_values).reset_index()

# Mantém apenas as colunas 'codigo_procedimento' e 'procedimento' e remove duplicatas
procedimentos_cnes_sem_nan = procedimentos_cnes_sem_nan[['codigo_procedimento', 'procedimento']].drop_duplicates()


In [28]:
procedimentos_cnes_sem_nan

,codigo_procedimento,procedimento
0,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS
1,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS
2,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...
3,303060107,TRATAMENTO DE CRISE HIPERTENSIVA
4,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...
...,...,...
54561,408060638,TRATAMENTO CIRÚRGICO DE LUXAÇÃO / FRATURA-LUXA...
54844,303080086,TRATAMENTO DE FARMACODERMIAS
55480,408030321,"ARTRODESE TORACO-LOMBO-SACRA POSTERIOR, SETE N..."
55482,408050667,TRATAMENTO CIRÚRGICO DE LESÃO AGUDA CAPSULO-LI...


In [29]:
procedimentos_cnes_validos = len(procedimentos_cnes_sem_nan)

In [49]:
procedimentos_cnes_sem_nan.to_excel('./data/procedimentos_cnes_iris.xlsx')

NOTA 2: isso implica que no total, seriam 921 os procedimentos POR CNES com no mínimo 90% de valores válidos (não nulos) nas linhas que poderiam ser analizados pela IRIS sem que seja necesário artifícios de tratamento e substituição de missings por dados sintéticos que causem viéses nos dados (substituição de missings por dados calculados, como média ou mediana. Sendo que, para a solução proposta conforme método de auditoria do MS, a mediana seria mais adequada para não repetir os viéses dos outliers de possíveis fraudes e porque é a medida utilizada nos processos de auditoria como no caso do Maranhão)

In [30]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_mun.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
procedimentos_mun_sem_nan = df_qtde_proc_mun.dropna(thresh=min_valid_values).reset_index()

# Mantém apenas as colunas 'codigo_procedimento' e 'procedimento' e remove duplicatas
procedimentos_mun_sem_nan = procedimentos_mun_sem_nan[['codigo_procedimento', 'procedimento']].drop_duplicates()

procedimentos_mun_sem_nan

,codigo_procedimento,procedimento
0,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS
1,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS
2,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...
3,303060107,TRATAMENTO DE CRISE HIPERTENSIVA
4,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...
...,...,...
48522,409060267,SALPINGOPLASTIA VIDEOLAPAROSCOPICA
48542,412030098,PLEUROSTOMIA
48733,408060409,RETIRADA DE TRAÇÃO TRANS-ESQUELÉTICA
52951,408030321,"ARTRODESE TORACO-LOMBO-SACRA POSTERIOR, SETE N..."


In [31]:
procedimentos_mun_validos = len(procedimentos_mun_sem_nan)
procedimentos_mun_validos

1176

In [48]:
procedimentos_mun_sem_nan.to_excel('./data/procedimentos_mun_iris.xlsx')

### df_proc_cnes é o dataframe com as quantidades de procedimentos POR CNES com menos de 10% de valores NaN nas linhas (ou seja, nos 13 anos ou 156 colunas de competencia mes/ano)
### Este df deve ser a base da qual calculamos todas as demais (df com as distâncias das medianas e assim por diante.. e os mesmos procedimentos devem ser feitos com os df com valores)

In [32]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_cnes.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
df_proc_cnes = df_qtde_proc_cnes.dropna(thresh=min_valid_values).reset_index()

df_proc_cnes = df_proc_cnes.drop('index', axis =1)

df_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,...,4.0,22.0,12.0,56.0,157.0,NaN,63.0,249.0,2.0,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.0,109.0,137.0,89.0,...,16.0,23.0,14.0,70.0,105.0,108.0,6.0,45.0,163.0,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.0,NaN,4.0,7.0,...,20.0,16.0,8.0,76.0,307.0,87.0,120.0,46.0,220.0,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.0,11.0,37.0,4.0,...,14.0,16.0,16.0,11.0,91.0,27.0,50.0,8.0,37.0,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.0,18.0,10.0,12.0,...,10.0,8.0,10.0,3.0,33.0,258.0,4.0,17.0,12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,16.0,84.0,65.0,72.0,...,63.0,63.0,31.0,63.0,63.0,NaN,NaN,NaN,13.0,38.0
55601,530010,BRASILIA,DF,3077098,70103014,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,14.0,36.0,92.0,30.0,...,27.0,27.0,27.0,27.0,27.0,NaN,NaN,NaN,6.0,19.0
55602,530010,BRASILIA,DF,3206874,50301001,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,NaN,62.0,77.0,80.0,...,62.0,82.0,70.0,99.0,53.0,62.0,29.0,66.0,40.0,NaN
55603,530010,BRASILIA,DF,3206874,50401002,PROCESSAMENTO DE CORNEA / ESCLERA,NaN,129.0,170.0,198.0,...,158.0,155.0,165.0,227.0,119.0,142.0,42.0,150.0,88.0,NaN


### df_proc_mun é o dataframe com as quantidades de procedimentos POR MUNICÍPIO com menos de 10% de valores NaN nas linhas (ou seja, nos 13 anos ou 156 colunas de competencia mes/ano)
### Este df deve ser a base da qual calculamos todas as demais (df com as distâncias das medianas, df com a quantidade de procedimentos para cada 10 mil hab. e assim por diante.. e os mesmos procedimentos devem ser feitos com os df com valores)

In [33]:
# Define o número mínimo de valores não nulos necessários para manter uma linha
n_cols = len(df_qtde_proc_mun.columns)
min_valid_values = int(n_cols * 0.9)

# Filtra o DataFrame para manter apenas as linhas com pelo menos 90% dos valores não nulos
df_proc_mun = df_qtde_proc_mun.dropna(thresh=min_valid_values).reset_index()

# drop coluna desnecassária criada nos procedimetnos anteriores
df_proc_mun = df_proc_mun.drop('index', axis =1)

# modifica tipo de variável para codigo do município para inteiro
df_proc_mun['codigo_municipio'] = df_proc_mun['codigo_municipio'].astype(int)

df_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,10.0,...,4.0,22.0,12.0,56.0,157.0,NaN,63.0,249.0,2.0,NaN
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.0,109.0,137.0,89.0,74.0,...,16.0,23.0,14.0,70.0,105.0,108.0,6.0,45.0,163.0,NaN
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.0,NaN,4.0,7.0,9.0,...,20.0,16.0,8.0,76.0,307.0,87.0,120.0,46.0,220.0,NaN
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.0,11.0,37.0,4.0,19.0,...,14.0,16.0,16.0,11.0,91.0,27.0,50.0,8.0,37.0,NaN
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.0,18.0,10.0,12.0,8.0,...,10.0,8.0,10.0,3.0,33.0,258.0,4.0,17.0,12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,87.5,29.0,16.0,46.3,49.5,...,34.0,43.2,19.8,17.8,27.5,16.0,27.8,123.3,35.0,47.8
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,6.0,NaN,56.5,4.0,23.0,...,76.6,34.0,55.5,40.4,29.8,64.6,48.8,123.0,59.5,47.7
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,16.0,184.0,13.0,26.0,57.0,...,78.0,69.0,27.2,117.7,53.0,149.5,70.7,52.0,81.0,26.7
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,31.0,9.0,47.5,20.5,NaN,...,67.5,116.0,59.5,65.5,55.0,29.0,39.0,84.5,145.5,5.0


In [34]:
# Selecionar apenas colunas numéricas após a quarta coluna
num_cols = df_proc_mun.columns[5:]

# Substituindo NaN pela mediana de cada linha
df_proc_mun[num_cols] = df_proc_mun[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)

# Calculando o desvio da mediana para cada célula
desvios_med_n_mun = df_proc_mun[num_cols].apply(lambda x: x - x.median(), axis=1)

# Criando um novo DataFrame com os desvios_med_n como uma matriz (m)
m_desvios_med_n_mun = pd.DataFrame(desvios_med_n_mun.values, index=df_proc_mun.index, columns=num_cols)

# Concatenando as quatro primeiras colunas do DataFrame original com o novo DataFrame de desvios_med_n_mun
df_desv_med_n_proc_mun = pd.concat([df_proc_mun.iloc[:, :5], m_desvios_med_n_mun], axis=1)
df_desv_med_n_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,-4.0,1.0,4.0,-2.0,2.0,...,-4.0,14.0,4.0,48.0,149.0,0.0,55.0,241.0,-6.0,0.0
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,79.0,72.0,100.0,52.0,37.0,...,-21.0,-14.0,-23.0,33.0,68.0,71.0,-31.0,8.0,126.0,0.0
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,-5.0,0.0,-13.0,-10.0,-8.0,...,3.0,-1.0,-9.0,59.0,290.0,70.0,103.0,29.0,203.0,0.0
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,-11.0,-9.0,17.0,-16.0,-1.0,...,-6.0,-4.0,-4.0,-9.0,71.0,7.0,30.0,-12.0,17.0,0.0
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,-4.0,4.0,-4.0,-2.0,-6.0,...,-4.0,-6.0,-4.0,-11.0,19.0,244.0,-10.0,3.0,-2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,37.5,-21.0,-34.0,-3.7,-0.5,...,-16.0,-6.8,-30.2,-32.2,-22.5,-34.0,-22.2,73.3,-15.0,-2.2
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,-29.0,0.0,21.5,-31.0,-12.0,...,41.6,-1.0,20.5,5.4,-5.2,29.6,13.8,88.0,24.5,12.7
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,-86.8,81.2,-89.8,-76.8,-45.8,...,-24.8,-33.8,-75.6,14.8,-49.8,46.7,-32.2,-50.8,-21.8,-76.2
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,-20.0,-42.0,-3.5,-30.5,0.0,...,16.5,65.0,8.5,14.5,4.0,-22.0,-12.0,33.5,94.5,-46.0


In [47]:
df_desv_med_n_proc_mun.to_excel('./data/df_desv_med_n_proc_mun.xlsx')

In [42]:
df_proc_mun[num_cols] = df_proc_mun[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)
df_proc_mun

,codigo_municipio,municipio,uf,codigo_procedimento,procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,10.0,...,4.0,22.0,12.0,56.0,157.0,8.0,63.0,249.0,2.0,8.0
1,110001,ALTA FLORESTA D'OESTE,RO,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.0,109.0,137.0,89.0,74.0,...,16.0,23.0,14.0,70.0,105.0,108.0,6.0,45.0,163.0,37.0
2,110001,ALTA FLORESTA D'OESTE,RO,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.0,17.0,4.0,7.0,9.0,...,20.0,16.0,8.0,76.0,307.0,87.0,120.0,46.0,220.0,17.0
3,110001,ALTA FLORESTA D'OESTE,RO,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.0,11.0,37.0,4.0,19.0,...,14.0,16.0,16.0,11.0,91.0,27.0,50.0,8.0,37.0,20.0
4,110001,ALTA FLORESTA D'OESTE,RO,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.0,18.0,10.0,12.0,8.0,...,10.0,8.0,10.0,3.0,33.0,258.0,4.0,17.0,12.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53028,530010,BRASILIA,DF,416120024,MASTECTOMIA RADICAL COM LINFADENECTOMIA AXILAR...,87.5,29.0,16.0,46.3,49.5,...,34.0,43.2,19.8,17.8,27.5,16.0,27.8,123.3,35.0,47.8
53029,530010,BRASILIA,DF,416120059,SEGMENTECTOMIA/QUADRANTECTOMIA/SETORECTOMIA DE...,6.0,35.0,56.5,4.0,23.0,...,76.6,34.0,55.5,40.4,29.8,64.6,48.8,123.0,59.5,47.7
53030,530010,BRASILIA,DF,503010014,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,16.0,184.0,13.0,26.0,57.0,...,78.0,69.0,27.2,117.7,53.0,149.5,70.7,52.0,81.0,26.7
53031,530010,BRASILIA,DF,505010097,TRANSPLANTE DE CORNEA,31.0,9.0,47.5,20.5,51.0,...,67.5,116.0,59.5,65.5,55.0,29.0,39.0,84.5,145.5,5.0


In [43]:
df_proc_mun.to_excel('./data/df_proc_mun.xlsx')

In [35]:
# Selecionar apenas colunas numéricas após a quarta coluna
num_cols = df_proc_cnes.columns[6:]

# Substituindo NaN pela mediana de cada linha
df_proc_cnes[num_cols] = df_proc_cnes[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)

# Calculando o desvio da mediana para cada célula
desvios_med_n_cnes = df_proc_cnes[num_cols].apply(lambda x: x - x.median(), axis=1)

# Criando um novo DataFrame com os desvios_med_n como uma matriz (m)
m_desvios_med_n_cnes = pd.DataFrame(desvios_med_n_cnes.values, index=df_proc_cnes.index, columns=num_cols)

# Concatenando as quatro primeiras colunas do DataFrame original com o novo DataFrame de desvios_med_n_cnes
df_desv_med_n_proc_cnes = pd.concat([df_proc_cnes.iloc[:, :5], m_desvios_med_n_cnes], axis=1)
df_desv_med_n_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,201001,201002,201003,201004,201005,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,-4.0,1.0,4.0,-2.0,2.0,...,-4.0,14.0,4.0,48.0,149.0,0.0,55.0,241.0,-6.0,0.0
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,79.0,72.0,100.0,52.0,37.0,...,-21.0,-14.0,-23.0,33.0,68.0,71.0,-31.0,8.0,126.0,0.0
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,-5.0,0.0,-13.0,-10.0,-8.0,...,3.0,-1.0,-9.0,59.0,290.0,70.0,103.0,29.0,203.0,0.0
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,-11.0,-9.0,17.0,-16.0,-1.0,...,-6.0,-4.0,-4.0,-9.0,71.0,7.0,30.0,-12.0,17.0,0.0
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,-4.0,4.0,-4.0,-2.0,-6.0,...,-4.0,-6.0,-4.0,-11.0,19.0,244.0,-10.0,3.0,-2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,-47.0,21.0,2.0,9.0,28.0,...,0.0,0.0,-32.0,0.0,0.0,0.0,0.0,0.0,-50.0,-25.0
55601,530010,BRASILIA,DF,3077098,70103014,-13.0,9.0,65.0,3.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,-8.0
55602,530010,BRASILIA,DF,3206874,50301001,0.0,-15.0,0.0,3.0,0.0,...,-15.0,5.0,-7.0,22.0,-24.0,-15.0,-48.0,-11.0,-37.0,0.0
55603,530010,BRASILIA,DF,3206874,50401002,0.0,-25.0,16.0,44.0,36.0,...,4.0,1.0,11.0,73.0,-35.0,-12.0,-112.0,-4.0,-66.0,0.0


In [46]:
df_desv_med_n_proc_cnes.to_excel('./data/df_desv_med_n_proc_cnes.xlsx')

In [44]:
df_proc_cnes[num_cols] = df_proc_cnes[num_cols].apply(lambda x: x.fillna(x.median()), axis=1)
df_proc_cnes

,codigo_municipio,municipio,uf,cnes,codigo_procedimento,procedimento,201001,201002,201003,201004,...,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212
0,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010037,TRATAMENTO DE OUTRAS DOENÇAS BACTERIANAS,4.0,9.0,12.0,6.0,...,4.0,22.0,12.0,56.0,157.0,8.0,63.0,249.0,2.0,8.0
1,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010061,TRATAMENTO DE DOENÇAS INFECCIOSAS INTESTINAIS,116.0,109.0,137.0,89.0,...,16.0,23.0,14.0,70.0,105.0,108.0,6.0,45.0,163.0,37.0
2,110001,ALTA FLORESTA D'OESTE,RO,2679477,303010193,TRATAMENTO DE OUTRAS DOENÇAS CAUSADAS POR VÍRU...,12.0,17.0,4.0,7.0,...,20.0,16.0,8.0,76.0,307.0,87.0,120.0,46.0,220.0,17.0
3,110001,ALTA FLORESTA D'OESTE,RO,2679477,303060107,TRATAMENTO DE CRISE HIPERTENSIVA,9.0,11.0,37.0,4.0,...,14.0,16.0,16.0,11.0,91.0,27.0,50.0,8.0,37.0,20.0
4,110001,ALTA FLORESTA D'OESTE,RO,2679477,303070064,TRATAMENTO DE DOENCAS DO ESOFAGO ESTOMAGO E DU...,10.0,18.0,10.0,12.0,...,10.0,8.0,10.0,3.0,33.0,258.0,4.0,17.0,12.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55600,530010,BRASILIA,DF,3077098,70103013,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,16.0,84.0,65.0,72.0,...,63.0,63.0,31.0,63.0,63.0,63.0,63.0,63.0,13.0,38.0
55601,530010,BRASILIA,DF,3077098,70103014,APARELHO DE AMPLIFICAÇÃO SONORA INDIVIDUAL (AA...,14.0,36.0,92.0,30.0,...,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,6.0,19.0
55602,530010,BRASILIA,DF,3206874,50301001,AÇÕES RELACIONADAS A DOAÇÃO DE ÓRGÃOS E TECIDO...,77.0,62.0,77.0,80.0,...,62.0,82.0,70.0,99.0,53.0,62.0,29.0,66.0,40.0,77.0
55603,530010,BRASILIA,DF,3206874,50401002,PROCESSAMENTO DE CORNEA / ESCLERA,154.0,129.0,170.0,198.0,...,158.0,155.0,165.0,227.0,119.0,142.0,42.0,150.0,88.0,154.0


In [45]:
df_proc_cnes.to_excel('./data/df_proc_cnes.xlsx')

### Calculando o df_proc_pop com os dados de quantidade de procedimentos para cada 10 mil habitantes para os 13 anos. 

In [36]:
df_proc_mun.to_excel('./data/df_proc_mun.xlsx')

In [41]:
# Convertendo o índice "date" em uma coluna
#df_pop = df_pop.reset_index()
# drop coluna desnecassária criada nos procedimetnos anteriores
#df_pop = df_pop.drop('index', axis =1)
df_pop

,codigo,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,1100015,24392,24228,24069,25728,25652,25578,25506,25437,23167,22945,22728,22516,22516
1,1100023,90353,91570,92747,101269,102860,104401,105896,107345,106168,107863,109523,111148,111148
2,1100031,6313,6221,6132,6495,6424,6355,6289,6224,5438,5312,5188,5067,5067
3,1100049,78574,78958,79330,85863,86556,87226,87877,88507,84813,85359,85893,86416,86416
4,1100056,17029,16939,16852,18041,18013,17986,17959,17934,16444,16323,16204,16088,16088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,12548,12644,12737,13227,13343,13456,13567,13675,13746,13863,13977,14088,14088
5566,5222054,7371,7475,7576,7933,8053,8171,8286,8397,8611,8743,8873,9002,9002
5567,5222203,4735,4846,4954,5246,5371,5495,5615,5731,6026,6171,6312,6451,6451
5568,5222302,5145,5195,5244,5460,5520,5578,5635,5690,5758,5821,5882,5941,5941
